# Load Packages

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import zipfile 

# Load DataFrame

In [ ]:
train = pd.read_csv("../input/histopathologic-cancer-detection/train_labels.csv", dtype=str)
print(train.shape)

In [ ]:
train.head(10)

# Label Distribution

In [ ]:
y_train = train.label

(train.label.value_counts() / len(train)).to_frame().T

# View Sample of Images

In [ ]:
plt.figure(figsize=(10,10)) 

for i in range(16):
    plt.subplot(4,4,i+1)   
    img = mpimg.imread(f'../input/histopathologic-cancer-detection/train/{train["id"][i]}.tif')
    plt.imshow(img)
    plt.text(0, -5, f'Label {train["label"][i]}')
    plt.axis('off')
    
plt.tight_layout()
plt.show()

# Split and Sample Data

In [ ]:
train_neg = train[train['label']=='0'].sample(10000,random_state=45)
train_pos = train[train['label']=='1'].sample(10000,random_state=45)

train_data = pd.concat([train_neg, train_pos], axis=0).reset_index(drop=True)

train = shuffle(train_data)

In [ ]:
train['label'].value_counts()

In [ ]:
def append_ext(fn):
    return fn+".tif"


train['id'] = train['id'].apply(append_ext)
train.head()

# Data Generators

In [ ]:
train_df, valid_df = train_test_split(train, test_size=0.2, random_state=45, stratify=train.label)

print(train_df.shape)
print(valid_df.shape)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
BATCH_SIZE = 64

train_loader = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = '../input/histopathologic-cancer-detection/train/',
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)

valid_loader = train_datagen.flow_from_dataframe(
    dataframe = valid_df,
    directory = '../input/histopathologic-cancer-detection/train/',
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (32,32)
)

In [ ]:
TR_STEPS = len(train_loader)
VA_STEPS = len(valid_loader)

print(TR_STEPS)
print(VA_STEPS)

# Build Network

In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=(32,32,3),
                                         include_top=False,
                                         weights='imagenet')

base_model.trainable = False

In [ ]:
base_model.summary()

In [ ]:
cnn1 = Sequential([
    base_model,
    
    Flatten(),
    
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.25),
    BatchNormalization(),
    Dense(2, activation='softmax')
])

cnn1.summary()

# Train Network

In [ ]:
opt = tf.keras.optimizers.Adam(0.001)
cnn1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

# Training Run 1

In [ ]:
%%time 

h1 = cnn1.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 15,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

In [ ]:
history = h1.history
print(history.keys())

In [ ]:
epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])

plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()

plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()

plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()

plt.tight_layout()
plt.show()

# Training Run 2

In [ ]:
tf.keras.backend.set_value(cnn1.optimizer.learning_rate, 0.0001)

In [ ]:
%%time 

h2 = cnn1.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 20,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

In [ ]:
for k in history.keys():
    history[k] += h2.history[k]

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()
plt.tight_layout()
plt.show()

# Training Run 3

In [ ]:
tf.keras.backend.set_value(cnn1.optimizer.learning_rate, 0.00001)

In [ ]:
%%time 

h3 = cnn1.fit(
    x = train_loader, 
    steps_per_epoch = TR_STEPS, 
    epochs = 30,
    validation_data = valid_loader, 
    validation_steps = VA_STEPS, 
    verbose = 1
)

In [ ]:
for k in history.keys():
    history[k] += h3.history[k]

epoch_range = range(1, len(history['loss'])+1)

plt.figure(figsize=[14,4])
plt.subplot(1,3,1)
plt.plot(epoch_range, history['loss'], label='Training')
plt.plot(epoch_range, history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,3,2)
plt.plot(epoch_range, history['accuracy'], label='Training')
plt.plot(epoch_range, history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.subplot(1,3,3)
plt.plot(epoch_range, history['auc'], label='Training')
plt.plot(epoch_range, history['val_auc'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
plt.legend()
plt.tight_layout()
plt.show()

# Save Model and History

In [ ]:
cnn1.save('cancer_model_v02.h5')
pickle.dump(history, open(f'cancer_history_v02.pkl', 'wb'))